# 🫰💵***Rob the casino with*** <img src='https://upload.wikimedia.org/wikipedia/commons/8/86/Google_JAX_logo.svg' alt="Environment" width="60" />

<div align="center">
<img src="https://images.unsplash.com/photo-1518895312237-a9e23508077d?auto=format&fit=crop&q=80&w=1784&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D" width=1000>
</div>

## **1) Q-learning for K-armed Bandits**

In [1]:
import jax
import jax.numpy as jnp
import pandas as pd
import plotly.express as px

from jax import random, lax, jit, vmap, pmap
from functools import partial
from jax_tqdm import loop_tqdm

import sys

sys.path.append("../../../")

from src import K_armed_bandits, SimpleBandit, BanditEpsilonGreedy

c:\Users\ryanp\AppData\Local\pypoetry\Cache\virtualenvs\jax-rl-KPtyfD6I-py3.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ryanp\AppData\Local\pypoetry\Cache\virtualenvs\jax-rl-KPtyfD6I-py3.10\lib\site-packages\jax\_src\api_util.py:172: SyntaxWarning: Jitted function has static_argnums=(0, 2), but only accepts 2 positional arguments. This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has {argnums_name}={argnums}, "


In [14]:
SEED = 0
K = 10

key = random.PRNGKey(SEED)

agent = SimpleBandit()
policy = BanditEpsilonGreedy(0.1)
env = K_armed_bandits(K, SEED)
env.render()

TIMESTEPS = 100_000

In [7]:
def rollout(key:random.PRNGKey, timesteps:int):
    @jit
    @loop_tqdm(timesteps)
    def fori_body(i:int, val:tuple):
        q_values, pulls, key = val
        action, key = policy(key, env.K, q_values)
        reward, key = env.get_reward(key, action)
        q_values, pulls = agent(action, reward, pulls, q_values)

        val = (q_values, pulls, key)

        return val

    q_values = jnp.zeros(env.K)
    pulls = jnp.zeros(env.K)  

    val_init = (q_values, pulls, key)
    val = lax.fori_loop(0, timesteps, fori_body, val_init)

    return val

In [15]:
q_values, pulls, _ = rollout(key, TIMESTEPS)

Running for 100,000 iterations: 100%|██████████| 100000/100000 [00:00<00:00, 2325582.32it/s]


In [25]:
px.imshow(jnp.array([q_values, env.bandits_q]))